<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220105_sm_nlp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model with only 2017 data

In [ ]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ds_3jm2n/kobert-tokenizer_4473a3c402354c5f98f5d7fa99136539
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ds_3jm2n/kobert-tokenizer_4473a3c402354c5f98f5d7fa99136539
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-n25zy_wv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-n25zy_wv


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib as plt

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (82,95,98,109,111,113,127,128,129,132,136) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (93,94,97,98,99,100,104,105,107,112,132,136,140,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (179) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (98,109,111,115,117,128,132) have mixed types.Specify dtype option on import or set 

In [ ]:
train3 = train3[train3['idx'] != 19871]

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1

cols = ['bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train1[col] = train1[col].map(null_to_no)

train1['text'] = train1[cols[0]] + ' ' + train1[cols[1]] + ' ' + train1[cols[2]] + ' ' + train1[cols[3]] + ' ' + train1[cols[4]] + ' ' + train1[cols[5]] + ' ' + train1[cols[6]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1

cols = ['bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train2[col] = train2[col].map(null_to_no)

train2['text'] = train2[cols[0]] + ' ' + train2[cols[1]] + ' ' + train2[cols[2]] + ' ' + train2[cols[3]] + ' ' + train2[cols[4]] + ' ' + train2[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1

cols = ['bq18_10',	'bq20_1',	'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train3[col] = train3[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train3[col]
  temp += ' '

train3['text'] = temp
# train3['text'] = train3[cols[0]] + ' ' + train3[cols[1]] + ' ' + train3[cols[2]] + ' ' + train3[cols[3]] + ' ' + train3[cols[4]] + ' ' + train3[cols[5]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1

cols = ['bq18_10',	'bq20_1',	'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train4[col] = train4[col].map(null_to_no)

train4['text'] = train4[cols[0]] + ' ' + train4[cols[1]] + ' ' + train4[cols[2]]

In [ ]:
df = pd.concat([train1[['idx', 'text', 'knowcode']], train2[['idx', 'text', 'knowcode']], train3[['idx', 'text', 'knowcode']], train4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text,knowcode
0,0,생산설비의 자동화로 없다 없다 없다 건설현장 노무직 없다 실업,825101
1,1,"건설 수주가 없어서 없다 매타기, 드라이버, 가위, CAD 없다 없다 없다 건축공학",140204
2,2,"신축 건설경기가 좋지 않아서 없다 건축설계표, 오토캐드, 파이프, 보일러 없다 없다...",140204
3,3,"업무량 감소, 자동화로 인해서 지적기사 토탈스테이션, SZP 무 지적공무원 없다 환경학과",140601
4,4,"부동산 거품 빠지면서 건축 경기도 덩달아 불황 없다 캐드, 엑셀, 건축도면 없다 없...",140204
...,...,...,...
35229,35244,없다 건설 경기의 어려움으로 다소 어려움 지구과학,121105
35230,35245,없다 광고 홍보 관련 시장이 켜져서 광고학,26102
35231,35246,없다 보험은 계속 필요 할 것이다 국문,31301
35232,35247,없다 인구수명이 길어지므로서 신약개발은 갈수록 많아질 것이라서 화학공학,154105


In [ ]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [ ]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [ ]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['knowcode'].map(target_to_label)

In [ ]:
dataset_train, dataset_test = train_test_split(df[['idx', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [ ]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
next(iter(data_train))

(array([   2, 4331, 7148, 3996, 7089, 3273,  517,  321, 6964, 7768, 3273,
        3273, 4660, 5452, 7821,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
len(df['target_label'].unique())

585

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train = []
test = []
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train.append(train_acc / (batch_id+1))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test.append(test_acc / (batch_id+1))
    # torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.351390838623047 train acc 0.0
epoch 1 batch id 201 loss 6.268958568572998 train acc 0.0024875621890547263
epoch 1 batch id 401 loss 6.293373107910156 train acc 0.0018703241895261845
epoch 1 batch id 601 loss 6.318737030029297 train acc 0.0014559068219633944
epoch 1 batch id 801 loss 6.453676223754883 train acc 0.001560549313358302
epoch 1 batch id 1001 loss 6.338784694671631 train acc 0.002247752247752248
epoch 1 batch id 1201 loss 6.368472099304199 train acc 0.0029142381348875937
epoch 1 batch id 1401 loss 6.378788471221924 train acc 0.003033547466095646
epoch 1 batch id 1601 loss 6.390063285827637 train acc 0.003279200499687695
epoch 1 batch id 1801 loss 6.404381275177002 train acc 0.003470294280955025
epoch 1 batch id 2001 loss 6.407719135284424 train acc 0.0036856571714142927
epoch 1 batch id 2201 loss 6.507796764373779 train acc 0.004600181735574738
epoch 1 batch id 2401 loss 6.5072479248046875 train acc 0.005414410662224073
epoch 1 batch id 2601 loss 6.1

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 1 test acc 0.0594697583914383


...

epoch 19 test acc 0.43903032268526027


  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.12803179025650024 train acc 1.0
epoch 20 batch id 201 loss 0.01502994168549776 train acc 0.9664179104477612
epoch 20 batch id 401 loss 0.10046319663524628 train acc 0.9669576059850374
epoch 20 batch id 601 loss 0.6833286881446838 train acc 0.96630615640599
epoch 20 batch id 801 loss 0.07644376158714294 train acc 0.9666042446941323
epoch 20 batch id 1001 loss 0.044848620891571045 train acc 0.9661588411588412
epoch 20 batch id 1201 loss 0.20978882908821106 train acc 0.9669025811823481
epoch 20 batch id 1401 loss 0.11837124824523926 train acc 0.9660064239828694
epoch 20 batch id 1601 loss 0.3042115271091461 train acc 0.9659587757651468
epoch 20 batch id 1801 loss 0.016260456293821335 train acc 0.9666157690172127
epoch 20 batch id 2001 loss 0.1546705961227417 train acc 0.9670789605197402
epoch 20 batch id 2201 loss 0.09956149756908417 train acc 0.9663221263062245
epoch 20 batch id 2401 loss 0.09447585791349411 train acc 0.9660037484381507
epoch 20 batch id 2601 l

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 20 test acc 0.43917220690773473


In [ ]:
torch.save(model, '/content/gdrive/MyDrive/know/model2.pt')

IsADirectoryError: ignored